# United Airlines SkyHack Hackathon

### Importing Basic Libraries

In [1]:
import numpy as np
import pandas as pd

### Importing Datasets

In [2]:
df_satisfaction = pd.read_csv('Datasets/Survey data_Inflight Satisfaction Score.csv')
df_satisfaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47074 entries, 0 to 47073
Data columns (total 31 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   flight_number                     47074 non-null  int64 
 1   origin_station_code               47074 non-null  object
 2   destination_station_code          47074 non-null  object
 3   record_locator                    47074 non-null  object
 4   scheduled_departure_date          47074 non-null  object
 5   question_text                     47074 non-null  object
 6   score                             47074 non-null  object
 7   satisfaction_type                 34963 non-null  object
 8   driver_sub_group1                 47074 non-null  object
 9   driver_sub_group2                 47074 non-null  object
 10  arrival_delay_minutes             47074 non-null  int64 
 11  arrival_delay_group               47074 non-null  object
 12  cabin_code_desc   

In [3]:
df_satisfaction.iloc[0]

flight_number                                                                    3802
origin_station_code                                                               MKX
destination_station_code                                                          ORX
record_locator                                                                 CYXXJJ
scheduled_departure_date                                                     9/1/2022
question_text                       How satisfied were you with the food & beverag...
score                                                                               2
satisfaction_type                                                        Dissatisfied
driver_sub_group1                                                     food & beverage
driver_sub_group2                                      food and beverage satisfaction
arrival_delay_minutes                                                             -24
arrival_delay_group                                   

In [4]:
# columnHeaders = df_satisfaction.columns

# for header in columnHeaders:
#     temp = f'{header}'
#     df_satisfaction[temp].info()
#     print(df_satisfaction[temp].value_counts(),"\n\n")

### Cleaning the table

In [5]:
# FILLING NULL VALUES WITH MOST RELEVANT VALUES

df_satisfaction.entity[df_satisfaction["entity"].isnull() == True] = 'Domestic'
df_satisfaction.loyalty_program_level[df_satisfaction["loyalty_program_level"].isnull() == True] = 'not-member'
df_satisfaction.media_provider[df_satisfaction["media_provider"].isnull() == True] = 'PANASONIC'
df_satisfaction.cabin_code_desc[df_satisfaction.cabin_name == 'Economy Plus'] = 'Economy Plus'


# REPLACING VALUES TO CREATE MORE MEANINGFULL CATEGORIES

df_satisfaction.arrival_delay_group[df_satisfaction.arrival_delay_minutes >= 120] = 'Long_Delay'          # 2+ hours
df_satisfaction.arrival_delay_minutes[df_satisfaction.arrival_delay_minutes >= 120] = None

df_satisfaction.arrival_delay_group[df_satisfaction.arrival_delay_minutes >= 60] = 'Medium_Delay'         # 1+ hours
df_satisfaction.arrival_delay_minutes[df_satisfaction.arrival_delay_minutes >= 60] = None

df_satisfaction.arrival_delay_group[df_satisfaction.arrival_delay_minutes >= 0] = 'Small_Delay'           # 0-1 hours
df_satisfaction.arrival_delay_minutes[df_satisfaction.arrival_delay_minutes >= 0] = None

df_satisfaction.arrival_delay_group[df_satisfaction.arrival_delay_minutes <= -40] = 'Very_Early'          # 30 min + hours
df_satisfaction.arrival_delay_minutes[df_satisfaction.arrival_delay_minutes <= -40] = None

df_satisfaction.arrival_delay_group[df_satisfaction.arrival_delay_minutes <= -20] = 'Medium_Early'          # 30 min + hours
df_satisfaction.arrival_delay_minutes[df_satisfaction.arrival_delay_minutes <= -20] = None

df_satisfaction.arrival_delay_group[df_satisfaction.arrival_delay_minutes < 0] = 'Small_Early'              # 0-30min early
df_satisfaction.arrival_delay_minutes[df_satisfaction.arrival_delay_minutes < 0] = None

C:\Users\jaing\AppData\Local\Temp\ipykernel_3548\4259318331.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_satisfaction.entity[df_satisfaction["entity"].isnull() == True] = 'Domestic'
C:\Users\jaing\AppData\Local\Temp\ipykernel_3548\4259318331.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_satisfaction.loyalty_program_level[df_satisfaction["loyalty_program_level"].isnull() == True] = 'not-member'
C:\Users\jaing\AppData\Local\Temp\ipykernel_3548\4259318331.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [6]:
# REMOVING OUTLYERS

indexes1 = np.where(df_satisfaction.loyalty_program_level == 'NBK')
indexes2 = np.where(df_satisfaction.generation == 'Greatest')
indexes3 = np.where(df_satisfaction.equipment_type_code == 'E7M')

arr = indexes1[0].tolist() + indexes2[0].tolist() + indexes3[0].tolist()

print(arr)

for i in reversed(range(0,len(arr))):
    df_satisfaction.drop(arr[i],axis=0,inplace=True)


[2390, 22231, 27848, 562, 43835, 43836, 43235]


In [7]:
# REMOVE
# actual_flown_miles         CAN BE USED FOR A MORE SPECIFIC EXAMINATION LATER
remove_columns = ["record_locator","driver_sub_group1", "arrival_delay_minutes", "cabin_name", "actual_flown_miles",
                  "departure_gate", "arrival_gate"]
df_satisfaction = df_satisfaction.drop(columns=remove_columns)

### Splitting tables by question type

In [8]:
a = df_satisfaction.driver_sub_group2 == 'food and beverage satisfaction'
df_q1 = df_satisfaction[a]
df_q2 = df_satisfaction[~a]

In [9]:
df_q1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34957 entries, 0 to 47071
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   flight_number                     34957 non-null  int64 
 1   origin_station_code               34957 non-null  object
 2   destination_station_code          34957 non-null  object
 3   scheduled_departure_date          34957 non-null  object
 4   question_text                     34957 non-null  object
 5   score                             34957 non-null  object
 6   satisfaction_type                 34957 non-null  object
 7   driver_sub_group2                 34957 non-null  object
 8   arrival_delay_group               34957 non-null  object
 9   cabin_code_desc                   34957 non-null  object
 10  entity                            34957 non-null  object
 11  number_of_legs                    34957 non-null  int64 
 12  seat_factor_band  

In [10]:
remove_columns = ["question_text", "driver_sub_group2"]
df_q1 = df_q1.drop(columns=remove_columns)

In [11]:
df_q1['score'].value_counts()

2    8250
1    7710
5    6733
3    6388
4    5876
Name: score, dtype: int64

### Splitting question 1 table by satisfaction vs disatisfaction

In [12]:
a = df_q1.satisfaction_type == 'Satisfied'
df_satisfied = df_q1[a]
df_dissatisfied = df_q1[~a]

# FOR LATER
y_train = df_q1['satisfaction_type']

In [13]:
remove_columns = ["satisfaction_type", "score"]
df_q1 = df_q1.drop(columns=remove_columns)
df_satisfied = df_satisfied.drop(columns=remove_columns)
df_dissatisfied = df_dissatisfied.drop(columns=remove_columns)

In [14]:
# columnHeaders = df_satisfied.columns

# for header in columnHeaders:
#     temp = f'{header}'
#     df_satisfaction[temp].info()
#     print(df_satisfaction[temp].value_counts(),"\n\n")

## Creating new dataframe to calculate factor-effect

In [30]:
imbalance_pos = df_satisfied.shape[0]/df_q1.shape[0]*100

head = []
factors = []
total_counts = []
pos_count = []
neg_count = []
factor_percent = []
pos_effect = []

columnHeaders = df_q1.columns
for header in columnHeaders:
    temp = f'{header}'
    arr = df_q1[temp].unique()
    for i in range(0,len(arr)):
        head.append(temp)
        
        factors.append(arr[i])
        
        len_factor0 = np.where(df_q1[temp] == arr[i])
        total_counts.append(len(len_factor0[0]))
        
        len_factor1 = np.where(df_satisfied[temp] == arr[i])
        pos_count.append(len(len_factor1[0]))
        
        len_factor2 = np.where(df_dissatisfied[temp] == arr[i])
        neg_count.append(len(len_factor2[0]))
        
        a = round((len(len_factor0[0])/df_q1.shape[0])*100,2)
        factor_percent.append(a)
        
        b = round((len(len_factor1[0])/len(len_factor0[0]))*100-imbalance_pos,2)
        pos_effect.append(round(a*b,2))
        
        

df_factor_effect = pd.DataFrame(np.column_stack([head,factors,total_counts,pos_count,neg_count,factor_percent,pos_effect]),
                               columns=['Head', 'Factors','Total Counts', 'Satisfied Count', 'Disatisfied Count', 
                                        'Factor Contribution Percent', 'Factor Affect'])


In [31]:
# REMOVING LOW AFFECTING FACTORS

indexes = []
low_affect = df_factor_effect['Factor Affect'].to_list()
for i in range(0,len(low_affect)):
    low_affect[i] = abs(float(low_affect[i]))
    if low_affect[i] <= 3:
        indexes.append(i)
for i in reversed(range(0,len(indexes))):
    df_factor_effect.drop(indexes[i],axis=0,inplace=True)

In [32]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

display(df_factor_effect)

,Head,Factors,Total Counts,Satisfied Count,Disatisfied Count,Factor Contribution Percent,Factor Affect
677,flight_number,384,67,12,55,0.19,-3.45
740,flight_number,115,51,4,47,0.15,-4.23
774,flight_number,363,87,17,70,0.25,-4.13
799,flight_number,985,104,25,79,0.3,-3.61
803,flight_number,86,100,17,83,0.29,-5.53
814,flight_number,83,41,2,39,0.12,-3.74
815,flight_number,84,71,13,58,0.2,-3.55
819,flight_number,956,77,15,62,0.22,-3.65
1364,flight_number,92,70,9,61,0.2,-4.64
1768,flight_number,980,51,31,20,0.15,3.71


In [24]:
df_factor_effect.to_csv(r'Datasets/Factors_Effect_Dataset.csv')

## From this point onwards, the analysis was done in excel

### G-Drive link: https://docs.google.com/spreadsheets/d/1KYWEmVz-TJAaF1vNqoRVMNemXV0HHkJpveUgmSQmKm8/edit#gid=0